# Third fine-tuning with a 0.5B parameter model, using `DataParallel`

My goal is to fine-tune meta-llama/Meta-Llama-3-8B on timdettmers/openassistant-guanaco.  This notebook reworks `second-0.5b-fine-tune.ipynb` so that it can use the `DataParallel` wrapper to tune over multiple GPUs.  It also strips out a lot of the investigatory stuff from that notebook.

## The dataset


In [1]:
dataset_source = "timdettmers/openassistant-guanaco"

In [2]:
from datasets import load_dataset

dataset = load_dataset(dataset_source)

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
import re

prompt_template = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{question} [/INST]
{response}
"""

pattern = r"### Human: (.*?)### Assistant: (.*)"

def rewrite_prompts(examples):
    questions = []
    responses = []
    # Iterate over each example
    for text in examples["text"]:
        match = re.search(pattern, text, re.DOTALL)
        if match:
            question = match.group(1).strip()
            response = match.group(2).strip()
            reformatted_text = prompt_template.format(question=question, response=response)
            while "### Human: " in reformatted_text:
                reformatted_text = reformatted_text.replace("### Human: ", "[INST]", 1)
                if "### Assistant: " in reformatted_text:
                    reformatted_text = reformatted_text.replace("### Assistant: ", "[/INST]\n", 1)
                else:
                    reformatted_text += "[/INST]\n"
                    
            responses.append(reformatted_text)
        else:
            # You might want to handle errors differently
            responses.append("Error: Did not match expected pattern.")
    return {"reformatted_text": responses}

# Apply the function to your dataset
reformatted_dataset = dataset.map(rewrite_prompts, batched=True)

## The model


In [4]:
base_model = "Qwen/Qwen1.5-0.5B"

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from torch.nn import DataParallel

parallel_model = DataParallel(model).cuda()

## Initial inference check

In [ ]:
tokenized_train_dataset = reformatted_dataset['train'].map(lambda row: tokenizer(row["reformatted_text"]))
tokenized_test_dataset = reformatted_dataset['test'].map(lambda row: tokenizer(row["reformatted_text"]))

In [ ]:
from transformers import TrainingArguments,Trainer

batch_size = 1
args = TrainingArguments(
    'outputs', 
    learning_rate=8e-5, 
    warmup_ratio=0.1, 
    lr_scheduler_type='cosine', 
    fp16=True,
    evaluation_strategy="epoch", 
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size * 2,
    num_train_epochs=2, 
    weight_decay=0.01, 
    report_to='none',
    remove_unused_columns=False,
)

In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(examples["reformatted_text"], truncation=True, padding="max_length", max_length=2048)
    tokenized["labels"] = tokenized["input_ids"][:]
    return tokenized

tokenized_dataset = reformatted_dataset.map(tokenize_function, batched=True)

In [ ]:
trainer = Trainer(
    parallel_model, args, 
    train_dataset=tokenized_dataset['train'], 
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

Turns out that it still starts getting worse validation loss after the second epoch -- so the instruction formatting didn't help with that.  But let's see what we get as a result when we try it!


In [ ]:
ask_question(trainer.model, "Who is Leonardo Da Vinci?")